# Working With ??

*TO DO*

## Preparing the Ground

As ever, we need to set up a database connection:

In [ ]:
from sqlite_utils import Database

# Open database connection
dbname = "all_openlean_xml.db"
db = Database(dbname)

And get a sample XML file, selecting one that we know contains struturally marked up glossary items:

In [ ]:
from lxml import etree
import pandas as pd

# Grab an OU-XML file that is known to contain glossary items
a210_xml_raw = pd.read_sql("SELECT xml FROM xml WHERE name='Approaching plays'",
                           con=db.conn).loc[0, "xml"]

# Parse the XML into an xml object
root = etree.fromstring(a210_xml_raw)

In [ ]:
import unicodedata
from lxml import etree

def unpack(x):
    return etree.tostring(x)

# via http://stackoverflow.com/questions/5757201/help-or-advice-me-get-started-with-lxml/5899005#5899005
def flatten(el):
    """Utility function for flattening XML tags."""
    def _flatten(el):
        if el is None:
            return ""  # Originally returned None; any side effects of move to ''?
        result = [(el.text or "")]
        for sel in el:
            result.append(_flatten(sel))
            result.append(sel.tail or "")
        return unicodedata.normalize("NFKD", "".join(result)) or " "
    return _flatten(el).strip()

In [ ]:
def fts(db, base_tbl, q):
    """Run a simple full-text search query 
       over a table with an FTS virtual table."""
    _q = f"""SELECT * FROM {base_tbl}_fts 
             WHERE {base_tbl}_fts MATCH {db.quote(q)} ;"""
    
    return pd.read_sql(_q, con=db.conn)